<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/Lottario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [117]:
df = pd.read_csv("/content/2023.csv")

In [118]:
df.head()

,PlayDate,N01,N02,N03,N04,N05,N06,N07
0,2001-09-01,2,3,7,9,34,45,16
1,2001-09-08,1,19,25,34,41,43,42
2,2001-09-15,2,22,23,29,30,32,40
3,2001-09-22,1,2,11,26,39,40,35
4,2001-09-29,7,14,23,27,38,41,20


In [179]:
df.tail()

,N01,N02,N03,N04,N05,N06,N07
1157,19,26,29,31,37,39,14
1158,3,5,12,25,38,39,10
1159,1,2,4,7,12,44,38
1160,9,19,29,31,42,43,28
1161,15,16,18,20,21,40,35


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  1162 non-null   object
 1   N01       1162 non-null   int64 
 2   N02       1162 non-null   int64 
 3   N03       1162 non-null   int64 
 4   N04       1162 non-null   int64 
 5   N05       1162 non-null   int64 
 6   N06       1162 non-null   int64 
 7   N07       1162 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 72.8+ KB


In [121]:
df.describe()

,N01,N02,N03,N04,N05,N06,N07
count,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000
mean,6.429432,13.290878,19.859725,26.641997,32.910499,39.511188,22.962134
std,5.112070,6.826361,7.581094,7.628151,6.796202,5.238034,12.986720
min,1.000000,2.000000,3.000000,5.000000,10.000000,19.000000,1.000000
25%,2.000000,8.000000,14.000000,21.000000,29.000000,37.000000,12.000000
50%,5.000000,13.000000,20.000000,27.000000,34.000000,41.000000,23.000000
75%,9.000000,18.000000,25.000000,32.000000,38.000000,44.000000,34.000000
max,32.000000,37.000000,42.000000,43.000000,44.000000,45.000000,45.000000


In [122]:
df.drop(['PlayDate'], axis=1, inplace=True)

In [123]:
df.head()

,N01,N02,N03,N04,N05,N06,N07
0,2,3,7,9,34,45,16
1,1,19,25,34,41,43,42
2,2,22,23,29,30,32,40
3,1,2,11,26,39,40,35
4,7,14,23,27,38,41,20


In [124]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [125]:
transformed_df.head()

,0,1,2,3,4,5,6
0,-0.866839,-1.508169,-1.697019,-2.313744,0.160379,1.048328,-0.536327
1,-1.062538,0.836695,0.678331,0.965001,1.190810,0.666341,1.466580
2,-0.866839,1.276357,0.414403,0.309252,-0.428438,-1.434588,1.312510
3,-1.062538,-1.654723,-1.169164,-0.084198,0.896401,0.093360,0.927336
4,0.111660,0.103925,0.414403,0.046952,0.749197,0.284354,-0.228188


In [126]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

1162

In [127]:
# Amount of games we need to take into consideration for prediction
window_length = 4
window_length

4

In [128]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

7

In [129]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
X

array([[[6.81530231e-310, 4.87710070e-310, 4.87710555e-310, ...,
         5.28543277e+180, 6.04799483e-067, 2.59759402e-057],
        [1.11565815e-047, 1.55790126e-076, 9.15566163e-072, ...,
         7.98253225e-072, 5.27366588e-091, 8.03971751e-096],
        [3.05438521e-057, 2.24626078e-057, 5.27366588e-091, ...,
         3.49992627e-033, 2.24594841e-057, 5.27366588e-091],
        [8.03967470e-096, 1.03914427e-095, 2.00171394e-052, ...,
         7.98470878e-072, 1.08596104e-095, 1.56941859e-076]],

       [[8.26108330e-072, 7.97918158e-072, 1.13296067e-095, ...,
         8.26108330e-072, 7.97918153e-072, 1.17959457e-095],
        [1.65578028e-076, 8.26108330e-072, 1.47210310e-052, ...,
         4.84950196e-245, 1.48339424e-076, 9.71939603e-072],
        [5.23214830e-067, 3.93594738e-245, 9.72157248e-072, ...,
         1.74214064e-076, 2.96938523e-260, 5.27485033e-091],
        [1.03914427e-095, 3.05437793e-057, 2.96621260e-260, ...,
         8.98693979e-096, 1.74214723e-076, 2.966608

In [130]:
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
y

array([[4.87710895e-310, 6.81530231e-310, 4.87710895e-310, ...,
        1.16633754e+200, 5.68175493e-322, 3.16202013e-322],
       [3.20633782e-319, 4.87710895e-310, 6.81530231e-310, ...,
        4.87710881e-310, 8.90020637e+252, 6.97283620e+228],
       [5.68175493e-322, 1.03259720e-321, 4.87710894e-310, ...,
        3.26083326e-322, 4.87711211e-310, 2.37151510e-322],
       ...,
       [1.34879921e-321, 4.87711211e-310, 2.42092166e-322, ...,
        6.81530231e-310, 2.32210854e-322, 1.11164770e-321],
       [4.74303020e-322, 3.16202013e-322, 4.87592731e-310, ...,
        3.45211701e+175, 4.24819625e+180, 4.13161109e-027],
       [6.81530231e-310, 5.88841767e-310, 3.21142670e-322, ...,
        6.81530231e-310, 3.45211701e+175, 4.24819625e+180]])

In [131]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [132]:
X.shape

(1158, 4, 7)

In [133]:
y.shape

(1158, 7)

In [134]:
X[0]

array([[-0.86683854, -1.5081693 , -1.69701917, -2.31374446,  0.16037927,
         1.04832766, -0.53632726],
       [-1.06253824,  0.83669469,  0.67833069,  0.96500074,  1.19080976,
         0.6663406 ,  1.46657993],
       [-0.86683854,  1.27635669,  0.41440293,  0.3092517 , -0.42843815,
        -1.43458825,  1.31251015],
       [-1.06253824, -1.6547233 , -1.16916364, -0.08419773,  0.89640105,
         0.09336001,  0.92733569]])

In [135]:
X[1]

array([[-1.06253824,  0.83669469,  0.67833069,  0.96500074,  1.19080976,
         0.6663406 ,  1.46657993],
       [-0.86683854,  1.27635669,  0.41440293,  0.3092517 , -0.42843815,
        -1.43458825,  1.31251015],
       [-1.06253824, -1.6547233 , -1.16916364, -0.08419773,  0.89640105,
         0.09336001,  0.92733569],
       [ 0.11165999,  0.1039247 ,  0.41440293,  0.04695208,  0.74919669,
         0.28435354, -0.22818769]])

In [136]:
y[0]

array([ 0.11165999,  0.1039247 ,  0.41440293,  0.04695208,  0.74919669,
        0.28435354, -0.22818769])

In [137]:
y[1]

array([-0.86683854,  0.39703269,  0.15047517,  1.48959997,  0.89640105,
        0.85733413,  0.7732659 ])

In [138]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 100

In [170]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [171]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [172]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=90)

# Train the model
history = model.fit(x=X, y=y, batch_size=100, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=1000, verbose=2)


Epoch 1/1000
12/12 - 16s - loss: 0.9994 - accuracy: 0.1442 - 16s/epoch - 1s/step
Epoch 2/1000
12/12 - 0s - loss: 0.9979 - accuracy: 0.1667 - 177ms/epoch - 15ms/step
Epoch 3/1000
12/12 - 0s - loss: 0.9962 - accuracy: 0.1554 - 161ms/epoch - 13ms/step
Epoch 4/1000
12/12 - 0s - loss: 0.9950 - accuracy: 0.1667 - 150ms/epoch - 13ms/step
Epoch 5/1000
12/12 - 0s - loss: 0.9940 - accuracy: 0.1598 - 155ms/epoch - 13ms/step
Epoch 6/1000
12/12 - 0s - loss: 0.9932 - accuracy: 0.1598 - 161ms/epoch - 13ms/step
Epoch 7/1000
12/12 - 0s - loss: 0.9921 - accuracy: 0.1693 - 154ms/epoch - 13ms/step
Epoch 8/1000
12/12 - 0s - loss: 0.9921 - accuracy: 0.1779 - 174ms/epoch - 15ms/step
Epoch 9/1000
12/12 - 0s - loss: 0.9911 - accuracy: 0.1667 - 154ms/epoch - 13ms/step
Epoch 10/1000
12/12 - 0s - loss: 0.9908 - accuracy: 0.1667 - 153ms/epoch - 13ms/step
Epoch 11/1000
12/12 - 0s - loss: 0.9898 - accuracy: 0.1615 - 151ms/epoch - 13ms/step
Epoch 12/1000
12/12 - 0s - loss: 0.9898 - accuracy: 0.1632 - 149ms/epoch - 12

In [173]:
loss, accuracy = model.evaluate(X, y, verbose=2)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

37/37 - 3s - loss: 0.0034 - accuracy: 0.9534 - 3s/epoch - 71ms/step
Test Loss: 0.0033752722665667534, Test Accuracy: 0.9533678889274597


In [143]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [209]:
df1 = df.copy()
to_predict = df.tail(4)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-209-4c852ff86c2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07
1158,3,5,12,25,38,39,10
1159,1,2,4,7,12,44,38
1160,9,19,29,31,42,43,28


In [210]:
to_predict = np.array(to_predict)
to_predict

array([[ 3,  5, 12, 25, 38, 39, 10],
       [ 1,  2,  4,  7, 12, 44, 38],
       [ 9, 19, 29, 31, 42, 43, 28]])

In [211]:
scaled_to_predict = scaler.transform(to_predict)

In [212]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 32ms/step
The predicted numbers in the last lottery game are: [ 4  9 17 21 28 36 33]


In [213]:
prediction = df.tail(1)
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])

The actual numbers in the last lottery game were: [15 16 18 20 21 40 35]


In [214]:
temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("winners:", duplicates)
print(len(duplicates)/number_of_features)

winners: [21]
0.14285714285714285


# Predict the next set of numbers

In [188]:
next = df.copy()
next = next.tail((window_length))
next

,N01,N02,N03,N04,N05,N06,N07
1158,3,5,12,25,38,39,10
1159,1,2,4,7,12,44,38
1160,9,19,29,31,42,43,28
1161,15,16,18,20,21,40,35


In [189]:
next = np.array(next)
next

array([[ 3,  5, 12, 25, 38, 39, 10],
       [ 1,  2,  4,  7, 12, 44, 38],
       [ 9, 19, 29, 31, 42, 43, 28],
       [15, 16, 18, 20, 21, 40, 35]])

In [152]:
x_next = scaler.transform(next)
x_next

next_Date = '2023-12-01 Evening';

In [153]:
y_next_pred = model.predict(np.array([x_next]))
print("The predicted numbers for the lottery game which will take place on",next_Date, "are (without rounding up):\n", scaler.inverse_transform(y_next_pred).astype(int)[0])

print("\n")
temp = np.array(scaler.inverse_transform(y_next_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("without rounding up")
print("winners:", duplicates)
print(len(duplicates)/20)

1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game which will take place on 2023-12-01 Evening are (without rounding up):
 [ 2  8 21 27 28 33  7]


without rounding up
winners: []
0.0


In [154]:
y_next_pred = model.predict(np.array([x_next]))
print("The predicted numbers for the lottery game which will take place on",next_Date, "are (with rounding up):\n", scaler.inverse_transform(y_next_pred).astype(int)[0]+1)

1/1 [==============================] - 0s 24ms/step
The predicted numbers for the lottery game which will take place on 2023-12-01 Evening are (with rounding up):
 [ 3  9 22 28 29 34  8]


In [155]:
temp = np.array(scaler.inverse_transform(y_next_pred).astype(int)[0]+1)
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("with rounding up")
print("winners:", duplicates)
print(len(duplicates)/20)

with rounding up
winners: [3]
0.05


In [219]:
to_predict_next = df.tail(4)
to_predict_next = np.array(to_predict_next)
to_predict_next

array([[ 3,  5, 12, 25, 38, 39, 10],
       [ 1,  2,  4,  7, 12, 44, 38],
       [ 9, 19, 29, 31, 42, 43, 28],
       [15, 16, 18, 20, 21, 40, 35]])

In [220]:
scaled_to_predict_next = scaler.transform(to_predict_next)
y_pred_next = model.predict(np.array([scaled_to_predict_next]))
print("The predicted numbers for the next draw are:\n", scaler.inverse_transform(y_pred_next).astype(int)[0])

1/1 [==============================] - 0s 22ms/step
The predicted numbers for the next draw are:
 [ 8 12 20 23 24 36 26]
